In [2]:
!pip install tensorflow==2.2.0

     |████████████████████████████████| 516.2MB 32kB/s 
     |████████████████████████████████| 460kB 51.4MB/s 
     |████████████████████████████████| 3.0MB 48.8MB/s 
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [7]:
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Activation, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from tensorflow import keras

import zipfile

In [8]:
print(tf.__version__)

2.2.0


In [9]:
zip_ref = zipfile.ZipFile("/content/drive/My Drive/ravdess/vid_frame.zip", 'r')
zip_ref.extractall("/content/emotion")
zip_ref.close()

In [10]:
TRAINING_DIR = "/content/emotion/training/"
VALIDATION_DIR = "/content/emotion/validation/"

In [11]:
# train_data_dir = 'drive/My Drive/Colab Notebooks/frames-only_face_dataset/'
batch_size = 64

train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.0,
    zoom_range=0.0,
    horizontal_flip=False)
#     validation_split=0.1) # set validation split

train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(227, 227),
    batch_size=batch_size,
    class_mode='categorical')
#     subset='training') # set as training data

val_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.0,
    zoom_range=0.0,
    horizontal_flip=False)

validation_generator = val_datagen.flow_from_directory(
    VALIDATION_DIR, 
    target_size=(227, 227),
    batch_size=batch_size,
    class_mode='categorical')
#     subset='validation') # set as validation data

Found 52314 images belonging to 8 classes.
Found 5550 images belonging to 8 classes.


In [12]:
print(train_generator)

In [18]:
#AlexNet
def createModel():

    input_shape = (227, 227, 3)
    num_classes = 8

    #Instantiation
    AlexNet = Sequential()

    AlexNet.add(Conv2D(96, kernel_size=(11,11), strides= 4,
                        padding= 'valid', activation= 'relu',
                        input_shape= input_shape,
                        kernel_initializer= 'he_normal'))
    AlexNet.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                              padding= 'valid', data_format= None))

    AlexNet.add(Conv2D(256, kernel_size=(5,5), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))
    AlexNet.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                              padding= 'valid', data_format= None)) 

    AlexNet.add(Conv2D(384, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

    AlexNet.add(Conv2D(384, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

    AlexNet.add(Conv2D(256, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

    AlexNet.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                              padding= 'valid', data_format= None))

    AlexNet.add(Flatten())
    AlexNet.add(Dense(4096, activation= 'relu'))
    AlexNet.add(Dense(4096, activation= 'relu'))
    AlexNet.add(Dense(1000, activation= 'relu'))
    AlexNet.add(Dense(num_classes, activation= 'softmax'))

    return AlexNet

In [23]:
model = createModel()

In [24]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 55, 55, 96)        34944     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 27, 27, 256)       614656    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 13, 13, 384)       885120    
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 13, 13, 384)       1327488   
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 13, 13, 256)      

In [25]:
#Creating 2nd model and training(fitting)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
 
batch_size = 128
_epochs = 100

# Fit the model on the batches generated by datagen.flow().
'''history = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = 15)'''

filepath = 'drive/My Drive/AlexNet.h5'
earlyStop = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=30)
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
callbacks_list = [earlyStop, checkpoint]

model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=_epochs,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // batch_size,
        callbacks=callbacks_list)

Epoch 1/100
408/408 [==============================] - ETA: 0s - loss: 2.1134 - accuracy: 0.2556
Epoch 00001: val_accuracy improved from -inf to 0.22057, saving model to drive/My Drive/AlexNet.h5
408/408 [==============================] - 69s 168ms/step - loss: 2.1134 - accuracy: 0.2556 - val_loss: 2.0724 - val_accuracy: 0.2206
Epoch 2/100
408/408 [==============================] - ETA: 0s - loss: 1.3952 - accuracy: 0.4719
Epoch 00002: val_accuracy improved from 0.22057 to 0.28052, saving model to drive/My Drive/AlexNet.h5
408/408 [==============================] - 72s 176ms/step - loss: 1.3952 - accuracy: 0.4719 - val_loss: 2.1053 - val_accuracy: 0.2805
Epoch 3/100
408/408 [==============================] - ETA: 0s - loss: 1.0166 - accuracy: 0.6287
Epoch 00003: val_accuracy improved from 0.28052 to 0.36301, saving model to drive/My Drive/AlexNet.h5
408/408 [==============================] - 72s 176ms/step - loss: 1.0166 - accuracy: 0.6287 - val_loss: 2.4092 - val_accuracy: 0.3630
Epoc

In [26]:
model.evaluate(validation_generator)

87/87 [==============================] - 11s 124ms/step - loss: 3.3402 - accuracy: 0.4234


[3.340196371078491, 0.4234234094619751]

In [28]:
TESTING_DIR = "/content/emotion/testing/"

test_datagen = ImageDataGenerator(rescale=1./255)

testing_generator = test_datagen.flow_from_directory(
    TESTING_DIR, # same directory as training data
    target_size= (227,227),
    batch_size= batch_size,
    class_mode='categorical')

model.evaluate(testing_generator)

Found 5528 images belonging to 8 classes.
44/44 [==============================] - 13s 287ms/step - loss: 4.3743 - accuracy: 0.3663


[4.374259948730469, 0.36631694436073303]

In [29]:
read_model = keras.models.load_model("/content/drive/My Drive/AlexNet.h5")

In [30]:
read_model.evaluate(testing_generator)

44/44 [==============================] - 10s 233ms/step - loss: 3.6840 - accuracy: 0.4175


[3.683985948562622, 0.41751086711883545]

In [32]:
read_model.evaluate(validation_generator)

87/87 [==============================] - 11s 121ms/step - loss: 2.5869 - accuracy: 0.4668


[2.586928367614746, 0.46684685349464417]